Import modules

In [2]:
import boto3
import json
import os
from dotenv import load_dotenv 
import cv2


Get environment variables and create AWS Session

In [3]:

dotenv_path = os.path.join("..", 'interface', '.env')
load_dotenv(dotenv_path)

#Connect to AWS Ressource, Credentials are optionals as we are accessing from the same AWS user
client=boto3.client('rekognition',aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("REGION_NAME"))
s3 = boto3.client('s3',
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("REGION_NAME")
)

#Define bucket name to access
bucket_name = os.getenv("BUCKET_NAME")

#Folder of real classroom photos
classroomFolder=r"C:\Users\User\Desktop\FinalProject\ImagesRekognition\RealClassroomPhotos"

Take all images of the folder and for each draw detected faces and save it to FacesDetectedInClassroom

In [ ]:

#Cycle that only draws rectangles where there are faces according to Rekognition's algorithm
i=1
boundingBoxDetectedFaces=[]

for file in os.listdir(classroomFolder):
    camera_local_file = rf'C:\Users\User\Desktop\FinalProject\ImagesRekognition\RealClassroomPhotos\{file}' 
    #convert files to bytes in order to be able to read it

    with open(camera_local_file, 'rb') as image_file:
        camera_file_bytes = image_file.read()
    responseDetection=client.detect_faces( Image={'Bytes': camera_file_bytes})
    
    # Read the image with OpenCV
    image = cv2.imread(camera_local_file)
    image_height, image_width, _ = image.shape

    boundingBoxDetectedFacesithImage=[]

    # Draw bounding boxes for each detected face
    for face in responseDetection['FaceDetails']:
        bbox = face['BoundingBox']
        left = int(bbox['Left'] * image_width)
        top = int(bbox['Top'] * image_height)
        width = int(bbox['Width'] * image_width)
        height = int(bbox['Height'] * image_height)
        cv2.rectangle(
            image, 
            (left, top), 
            (left + width, top + height), 
            (37, 173, 250), 
            2  
        )
        boundingBoxDetectedFacesithImage.append((left, top, width,height))
    #Each array element represents a sub-array of tuples containing the bonding boxes    
    boundingBoxDetectedFaces.append(boundingBoxDetectedFacesithImage) 
    # Print number of faces detected
    print(f"{len(responseDetection['FaceDetails'])} Faces were detected")

    # Save the annotated image
    output_path = rf"C:\Users\User\Desktop\FinalProject\ImagesRekognition\FacesDetectedInClassroom\faces_detected_{i}.jpeg"
    cv2.imwrite(output_path, image)
    #Increase the photo's index
    i=i+1


21 Faces were detected
15 Faces were detected


Take all images of the folder and all student's personal image and draw a unique color rectangle if recognized

In [97]:
student_array=["AlejandroCallasas", "AndresAnillo","DiegoGomez","GabrielCastanez", "MauricioDeLaHoz","YordiRochel", "MariaBerdugo","AlbaQuiroz","GiohanOlivares","JesusCotes"]
numberOfStudents=len(student_array)
colors=[(0, 200, 0),(200,0,0),(0,0,0), (0,0,255), (150,150,0),(0,150,150),(100,20,60),(20,60,100),(60,100,20),(255,255,255)]

#Array that contains all faces recognized in all test images. Each photo corresponds to the n-th array element
boundingBoxRecognizedFaces=[]

m=1
#Create image of the class with all faces recognized in it
for file in os.listdir(classroomFolder):
    camera_local_file = rf'C:\Users\User\Desktop\FinalProject\ImagesRekognition\RealClassroomPhotos\{file}' 

    #convert files to bytes in order to be able to read it
    with open(camera_local_file, 'rb') as image_file:
        camera_file_bytes = image_file.read()
    responseDetection=client.detect_faces( Image={'Bytes': camera_file_bytes})
    image = cv2.imread(camera_local_file)

    k=0
    recognized_counter=0

    boundingBoxRecognizedFacesithImage=[]

    for student_name in student_array:
        student_found = False  # Bandera para controlar si el estudiante fue encontrado
        
        student_id_photo=r"C:\Users\User\Desktop\FinalProject\ImagesRekognition\Student"+ rf"s\{student_name}.jpeg"
        with open(student_id_photo, 'rb') as image_file:
            student_id_photo_bytes = image_file.read()

        #Look for the face in SourceImage evaluating all the faces in TargetImage
        responseRecognition=client.compare_faces(
            SourceImage={'Bytes': student_id_photo_bytes},
            TargetImage={'Bytes': camera_file_bytes},
            SimilarityThreshold=80
        )
        # Read the target image with OpenCV

        # Get image dimensions
        image_height, image_width, _ = image.shape

        # Draw bounding boxes for matched faces
        for faceMatch in responseRecognition['FaceMatches']:
            # Get bounding box coordinates
            bbox = faceMatch['Face']['BoundingBox']
            
            # Convert normalized coordinates to pixel coordinates
            left = int(bbox['Left'] * image_width)
            top = int(bbox['Top'] * image_height)
            width = int(bbox['Width'] * image_width)
            height = int(bbox['Height'] * image_height)
        
            # Draw green rectangle (to differentiate from previous red detection boxes)
            cv2.rectangle(
                image, 
                (left, top), 
                (left + width, top + height), 
                colors[k],  # Green color in BGR
                2,
            )
            
            padding = 5  # Espacio entre el rectángulo y el cuadro de texto
            text_box_height = 25  # Altura del cuadro de texto
            # Agregar texto dentro del recuadro de texto
            cv2.putText(image, f"{student_name}", (left + 10, top - padding - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[k], 1)

            recognized_counter=recognized_counter+1
            print('Face at coordinates ' + str(bbox) + ' matches with ' + str(faceMatch['Similarity']) + '% confidence'+ f"and is likely {student_name}")

            #Save the identity and region as a tuple in the n-th array element that corresponds to the n-th image
            boundingBoxRecognizedFacesithImage.append((student_name,left,top,width,height))
            student_found = True  # Marcamos que este estudiante fue encontrado
        
        # Si no se encontró al estudiante, agregar una tupla con valores cero
        if not student_found:
            boundingBoxRecognizedFacesithImage.append((student_name, 0, 0, 0, 0))
            print(f"Student {student_name} not found in the image")
            
        k=k+1
    boundingBoxRecognizedFaces.append(boundingBoxRecognizedFacesithImage)

    # Save the annotated image
    output_path = rf"C:\Users\User\Desktop\FinalProject\ImagesRekognition\StudentFoundInClassroom\faces_in_classroom_{m}_{recognized_counter}.jpeg"
    cv2.imwrite(output_path, image)
    print(f"Annotated image saved to {output_path}")
    m=m+1

Face at coordinates {'Width': 0.02335977554321289, 'Height': 0.039030011743307114, 'Left': 0.5729531049728394, 'Top': 0.41381317377090454} matches with 99.63961029052734% confidenceand is likely AlejandroCallasas
Face at coordinates {'Width': 0.026119183748960495, 'Height': 0.039969634264707565, 'Left': 0.1270674467086792, 'Top': 0.50046706199646} matches with 93.06892395019531% confidenceand is likely AndresAnillo
Student DiegoGomez not found in the image
Face at coordinates {'Width': 0.025718938559293747, 'Height': 0.04472402110695839, 'Left': 0.013418024405837059, 'Top': 0.5241779088973999} matches with 89.78019714355469% confidenceand is likely GabrielCastanez
Face at coordinates {'Width': 0.017744731158018112, 'Height': 0.02946370467543602, 'Left': 0.5593912601470947, 'Top': 0.36348745226860046} matches with 86.79216766357422% confidenceand is likely MauricioDeLaHoz
Face at coordinates {'Width': 0.02150445058941841, 'Height': 0.035746827721595764, 'Left': 0.2898186147212982, 'Top'

In [ ]:
responseDetection

Obtain tuples from base line (Manual correction and verification)

In [98]:
true=True
faceGallery=[
{"classroom1.jpeg156424":{"filename":"classroom1.jpeg","size":156424,"regions":[{"shape_attributes":{"name":"rect","x":733,"y":392,"width":31,"height":44},"region_attributes":{"name":"AlejandroCallasas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":153,"y":475,"width":45,"height":46},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":660,"y":275,"width":20,"height":27},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":13,"y":506,"width":37,"height":39},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":716,"y":348,"width":22,"height":30},"region_attributes":{"name":"MauricioDeLaHoz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":367,"y":421,"width":33,"height":39},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1155,"y":362,"width":31,"height":35},"region_attributes":{"name":"MariaBerdugo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":908,"y":346,"width":27,"height":31},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1188,"y":326,"width":32,"height":36},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":114,"y":534,"width":44,"height":59},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}},
{"classroom2.jpeg148639":{"filename":"classroom2.jpeg","size":148639,"regions":[{"shape_attributes":{"name":"rect","x":147,"y":465,"width":56,"height":60},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":2,"y":496,"width":51,"height":64},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1206,"y":314,"width":46,"height":61},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":154,"y":551,"width":55,"height":74},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1158,"y":347,"width":44,"height":60},"region_attributes":{"name":"MariaBerdugo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":408,"y":415,"width":44,"height":59},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
]

m=1
#The N-Element contains tuples of the N-Th image with informations of students to be recognized
tuples=[]
for element in faceGallery:
    i=1
    #(f"photo {m}-----------------------------------------")
    filteredFaceGalleryObject=element[next(iter(element))]["regions"]
    tuplesnthPhoto=[]
    for element in filteredFaceGalleryObject:
        #print(f"Person {i}-----")
        region=element["shape_attributes"]
        identity=element["region_attributes"]
        #print(f"x = {region["x"]} \ny = {region["y"]} \nwidth = {region["width"]}\nheight = {region["height"]} \nidentifier = {identity["name"]}")
        tuplesnthPhoto.append((identity["name"],region["x"],region["y"], region["width"], region["height"]))
        i=i+1
    tuples.append(tuplesnthPhoto)
    m=m+1


Code to calculate IoU

In [95]:

def calculate_iou(box1, box2):
    """
    Calculate IoU between 2 boxes.
    box1, box2: tuples like (x, y, width, height)
    return IoU value that ranges from 0 to 1
    """
    # Extraer coordenadas
    name1,x1, y1, w1, h1 = box1
    name2, x2, y2, w2, h2 = box2
    
    # Calcular coordenadas de la intersección
    x_intersect_1 = max(x1, x2)
    y_intersect_1 = max(y1, y2)
    x_intersect_2 = min(x1 + w1, x2 + w2)
    y_intersect_2 = min(y1 + h1, y2 + h2)
    
    # Comprobar si hay intersección
    if x_intersect_2 <= x_intersect_1 or y_intersect_2 <= y_intersect_1:
        return 0.0
    
    # Calcular áreas
    area_intersect = (x_intersect_2 - x_intersect_1) * (y_intersect_2 - y_intersect_1)
    area_box1 = w1 * h1
    area_box2 = w2 * h2
    
    # Calcular unión
    area_union = area_box1 + area_box2 - area_intersect
    
    # Calcular IoU
    iou = area_intersect / area_union
    
    return iou


Iterate through the base line and compare it to the results obtained by Rekognition. Then calculate TP, TN, FP, FN

In [100]:
#It searchs based on face gallery/Base Line: It compares my definition of the existance of specific students and compares it to the found studens
#The result wouldn't be affected if the search direction is reversed

i=0
#Loop through all the photos
for photoObject in tuples:
    print(f"Results for photo {i+1}")
    TP=0
    TN=0
    FP=0
    FN=0
    #Obtain the array of faces recognized with Rekognition in that photo
    tuplesToCompare=boundingBoxRecognizedFaces[i]
    #Look for every bounding box drawn in the rekognition array. Element corresponds to the drawn rectangle, identify to rekognition's precdiction
    for element in photoObject:
        #print(element)
        for identity in tuplesToCompare:
            IoU=calculate_iou(element,identity)
            #Case True Positive: Identity and bounding box match
            #print(f"{element} compared with {identity}")

            if(IoU>=0.5 and element[0]==identity[0]):
                TP=TP+1

            #Case False Positive: no identity match, however, the bounding boxes are similar
            if(IoU>=0.5 and element[0]!=identity[0]):
                FP=FP+1

            #Case True Negative: No bounding box match. However, the identity is different
            if(IoU<0.5 and element[0]!=identity[0]):
                TN=TN+1

            #Case False Negative: The identity matches. However, bonding boxes do not correspond
            if(IoU<0.5 and element[0]==identity[0]):
                FN=FN+1 

    print(f"TP : {TP} \nTN : {TN} \nFP : {FP} \nFN : {FN}")
    i=i+1

Results for photo 1
TP : 9 
TN : 90 
FP : 0 
FN : 1
Results for photo 2
TP : 0 
TN : 54 
FP : 0 
FN : 6
